# **Random Forest**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 2 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/NotAnnotated/Regular/NotAveragePooled/split_features_1s_all_2D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [5]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [6]:
train_data

{'melspectrogram': array([[[-1.7363867e-02,  0.0000000e+00, -8.1198728e-03, ...,
          -9.7965123e-03, -4.7687645e-05, -1.3045984e-02],
         [-1.9384871e-01, -2.1228047e-01, -2.6948744e-01, ...,
          -2.7984515e-01, -2.1688208e-01, -1.9360192e-01],
         [-2.6834840e-01, -2.8662941e-01, -3.4242344e-01, ...,
          -3.5257360e-01, -2.9082543e-01, -2.6788574e-01],
         ...,
         [-9.9897534e-01, -9.9884158e-01, -9.9602479e-01, ...,
          -9.7042859e-01, -9.8348904e-01, -9.8610806e-01],
         [-9.9957860e-01, -9.9945968e-01, -9.9773401e-01, ...,
          -9.7953582e-01, -9.8669928e-01, -9.8847580e-01],
         [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, ...,
          -1.0000000e+00, -1.0000000e+00, -1.0000000e+00]],
 
        [[-1.7287681e-02,  0.0000000e+00, -8.1161708e-03, ...,
          -9.7880140e-03, -4.7853238e-05, -1.3035558e-02],
         [-1.9298783e-01, -2.1179421e-01, -2.6936716e-01, ...,
          -2.7959031e-01, -2.1686108e-01, -1.934

In [7]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [8]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [9]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [10]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [11]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [12]:
display(train_y.shape)
display(train_y[:15])

(42578,)

array([ 7, 16, 19,  0,  1,  7,  6,  3,  1,  1, 19,  7,  9,  7, 12],
      dtype=int32)

In [13]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(42578, 128, 87)

array([[-0.01625595,  0.        , -0.01081104, ..., -0.01750798,
        -0.00961135, -0.02381007],
       [-0.1902909 , -0.20826001, -0.26494688, ..., -0.28861395,
        -0.22765526, -0.2048849 ],
       [-0.2696766 , -0.28877833, -0.3450681 , ..., -0.35860714,
        -0.29948115, -0.27753055],
       ...,
       [-0.998848  , -0.99864674, -0.99782616, ..., -0.99787885,
        -0.99819416, -0.998252  ],
       [-0.9996349 , -0.99954635, -0.9991726 , ..., -0.9999292 ,
        -1.        , -1.        ],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -0.9998773 , -0.99985075]], dtype=float32)

'mfcc'

(42578, 20, 87)

...

(42578, 84, 87)

array([[-5.7999372e-02,  0.0000000e+00, -4.9227238e-02, ...,
        -6.4293253e-01, -5.5793744e-01, -5.9717023e-01],
       [-8.2591659e-01, -8.7020481e-01, -9.7758025e-01, ...,
        -3.6687988e-01, -3.2261634e-01, -3.4996593e-01],
       [-1.9113955e+00, -2.0960922e+00, -2.5002358e+00, ...,
        -1.2468940e+00, -1.0981936e+00, -1.1525719e+00],
       ...,
       [-2.0504864e+01, -6.2051636e+01, -5.6007896e+01, ...,
        -6.2008591e+01, -6.3932381e+01, -3.0597172e+01],
       [-2.0726175e+01, -6.2380585e+01, -5.9756584e+01, ...,
        -8.0000000e+01, -7.5222946e+01, -3.2498779e+01],
       [-2.0949743e+01, -6.9877197e+01, -6.2453812e+01, ...,
        -5.9540771e+01, -6.3284271e+01, -3.3768303e+01]], dtype=float32)

'id'

(42578,)

'XC184429.mp3'

In [14]:
display(val_y.shape)
display(val_y[:15])

(13083,)

array([ 3,  5, 11, 11, 18, 12, 19, 18,  1,  2,  1,  4, 17,  0, 11],
      dtype=int32)

In [15]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(13083, 128, 87)

array([[-1.7357247e-02,  0.0000000e+00, -8.1163393e-03, ...,
        -9.8069618e-03, -4.4781682e-05, -1.3083253e-02],
       [-1.9378066e-01, -2.1221067e-01, -2.6939127e-01, ...,
        -2.8022012e-01, -2.1750733e-01, -1.9421951e-01],
       [-2.6825911e-01, -2.8654340e-01, -3.4231266e-01, ...,
        -3.5307595e-01, -2.9167926e-01, -2.6874796e-01],
       ...,
       [-9.9951565e-01, -9.9949867e-01, -9.9945468e-01, ...,
        -9.9845719e-01, -9.9848551e-01, -9.9847937e-01],
       [-1.0000000e+00, -1.0000000e+00, -9.9998677e-01, ...,
        -9.9909991e-01, -9.9910897e-01, -9.9910593e-01],
       [-9.9996382e-01, -9.9996173e-01, -1.0000000e+00, ...,
        -1.0000000e+00, -1.0000000e+00, -1.0000000e+00]], dtype=float32)

'mfcc'

(13083, 20, 87)

...

(13083, 84, 87)

array([[-2.1184500e-01, -2.4014600e-01, -3.8562453e-01, ...,
        -1.5154918e-01, -1.4830172e-02,  2.9802322e-08],
       [-4.3986446e-01, -5.1669687e-01, -6.6346020e-01, ...,
        -3.9817017e-01, -2.6154357e-01, -2.0144252e-01],
       [-7.5364864e-01, -7.7070826e-01, -9.7469288e-01, ...,
        -6.9044250e-01, -4.7974968e-01, -4.8022574e-01],
       ...,
       [-1.9936699e+01, -6.5170654e+01, -5.5588852e+01, ...,
        -5.7528671e+01, -6.7710899e+01, -2.9000002e+01],
       [-2.0089506e+01, -6.0576035e+01, -4.9275063e+01, ...,
        -5.7286602e+01, -5.5994537e+01, -3.1073345e+01],
       [-2.0347136e+01, -5.5362125e+01, -5.1570663e+01, ...,
        -5.7042725e+01, -5.6348019e+01, -3.4093269e+01]], dtype=float32)

'id'

(13083,)

'XC511677.mp3'

# **Random Forest Model**

Saving evaluation results

In [16]:
def evaluate_model(rf, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = rf.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = rf.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [17]:
train_results = {}
val_results = {}

val_scores = {}

### **With all the features**

In [18]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(42578, 21228)

In [19]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(13083, 21228)

Fit the model with training data

In [20]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [21]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['all_features'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['all_features'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.250936329588015


In [22]:
val_scores['all_features'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.15      0.29      0.20       793
           1       0.47      0.56      0.51      1294
           2       0.12      0.31      0.17       617
           3       0.23      0.30      0.26       984
           4       0.06      0.01      0.02       229
           5       0.00      0.00      0.00       303
           6       0.00      0.00      0.00       275
           7       0.34      0.38      0.35       698
           8       0.14      0.54      0.22       176
           9       0.09      0.24      0.13       227
          10       0.25      0.12      0.16       455
          11       0.44      0.01      0.02      1660
          12       0.46      0.35      0.40      1261
          13       0.26      0.05      0.08       502
          14       0.04      0.08      0.05       156
          15       0.13      0.19      0.16       798
          16       0.68      0.62      0.65   

### **With MFCCs only**

In [23]:
training_features_3D = train_features['mfcc']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(42578, 1740)

In [24]:
validation_features_3D = val_features['mfcc']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(13083, 1740)

In [25]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [26]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.31674692348849653


In [27]:
val_scores['mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.32      0.66      0.43       793
           1       0.53      0.53      0.53      1294
           2       0.22      0.58      0.32       617
           3       0.55      0.66      0.60       984
           4       0.01      0.01      0.01       229
           5       0.06      0.04      0.05       303
           6       0.00      0.00      0.00       275
           7       0.29      0.52      0.37       698
           8       0.10      0.13      0.11       176
           9       0.38      0.32      0.35       227
          10       0.12      0.05      0.07       455
          11       0.18      0.01      0.01      1660
          12       0.48      0.43      0.45      1261
          13       0.22      0.07      0.10       502
          14       0.02      0.05      0.03       156
          15       0.10      0.24      0.14       798
          16       0.65      0.45      0.53   

### **With Mel-Spectrogram Only**

In [28]:
training_features_3D = train_features['melspectrogram']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(42578, 11136)

In [29]:
validation_features_3D = val_features['melspectrogram']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(13083, 11136)

In [30]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [31]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 0.9999765136925173
Validation accuracy: 0.2258656271497363


In [32]:
val_scores['melspectrogram'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.14      0.26      0.18       793
           1       0.38      0.53      0.44      1294
           2       0.16      0.31      0.21       617
           3       0.20      0.28      0.23       984
           4       0.00      0.00      0.00       229
           5       0.00      0.00      0.00       303
           6       0.00      0.00      0.00       275
           7       0.33      0.36      0.34       698
           8       0.16      0.53      0.25       176
           9       0.07      0.22      0.11       227
          10       0.30      0.18      0.22       455
          11       0.21      0.01      0.01      1660
          12       0.32      0.20      0.25      1261
          13       0.42      0.08      0.13       502
          14       0.02      0.04      0.03       156
          15       0.14      0.26      0.18       798
          16       0.57      0.49      0.53   

## **With Chroma Only**

In [33]:
training_features_3D = train_features['chroma']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(42578, 1044)

In [34]:
validation_features_3D = val_features['chroma']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(13083, 1044)

In [35]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [36]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.20943208744171826


In [37]:
val_scores['chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.10      0.54      0.16       793
           1       0.25      0.34      0.29      1294
           2       0.21      0.67      0.32       617
           3       0.59      0.27      0.37       984
           4       0.50      0.00      0.01       229
           5       0.00      0.00      0.00       303
           6       0.00      0.00      0.00       275
           7       0.12      0.33      0.18       698
           8       0.00      0.00      0.00       176
           9       0.40      0.20      0.27       227
          10       0.11      0.05      0.07       455
          11       0.00      0.00      0.00      1660
          12       0.33      0.06      0.11      1261
          13       0.00      0.00      0.00       502
          14       0.03      0.01      0.02       156
          15       0.45      0.66      0.54       798
          16       0.49      0.21      0.30   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC Score: 0.6770122623164095
F1 Score (Weighted): 0.17275680606885402


## **With CQT Only**

In [38]:
training_features_3D = train_features['cqt']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(42578, 7308)

In [39]:
validation_features_3D = val_features['cqt']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(13083, 7308)

In [40]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [41]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.18841244362913706


In [42]:
val_scores['cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.07      0.13      0.09       793
           1       0.42      0.42      0.42      1294
           2       0.09      0.29      0.14       617
           3       0.14      0.24      0.18       984
           4       0.03      0.00      0.01       229
           5       0.00      0.00      0.00       303
           6       0.00      0.00      0.00       275
           7       0.23      0.39      0.29       698
           8       0.20      0.42      0.27       176
           9       0.01      0.01      0.01       227
          10       0.00      0.00      0.00       455
          11       0.05      0.00      0.00      1660
          12       0.29      0.30      0.30      1261
          13       0.27      0.04      0.06       502
          14       0.02      0.04      0.03       156
          15       0.10      0.12      0.11       798
          16       0.65      0.46      0.54   

## **With MFCCs and Mel-Spectrogram**

In [43]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(42578, 12876)

In [44]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(13083, 12876)

In [45]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [46]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2520828556141558


In [47]:
val_scores['mfcc_melspectrogram'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.17      0.37      0.24       793
           1       0.42      0.53      0.47      1294
           2       0.12      0.29      0.17       617
           3       0.27      0.32      0.29       984
           4       0.06      0.02      0.03       229
           5       0.00      0.00      0.00       303
           6       0.00      0.00      0.00       275
           7       0.34      0.42      0.38       698
           8       0.17      0.54      0.25       176
           9       0.08      0.23      0.12       227
          10       0.28      0.19      0.22       455
          11       0.47      0.01      0.02      1660
          12       0.41      0.28      0.34      1261
          13       0.37      0.07      0.12       502
          14       0.03      0.08      0.05       156
          15       0.17      0.26      0.20       798
          16       0.63      0.54      0.58   

## **With MFCCs and Chroma**

In [48]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(42578, 2784)

In [49]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(13083, 2784)

In [50]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [51]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.31445387143621495


In [52]:
val_scores['mfcc_chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.31      0.66      0.43       793
           1       0.56      0.54      0.55      1294
           2       0.23      0.65      0.34       617
           3       0.52      0.64      0.57       984
           4       0.00      0.00      0.00       229
           5       0.06      0.05      0.05       303
           6       0.09      0.00      0.01       275
           7       0.28      0.47      0.35       698
           8       0.09      0.12      0.11       176
           9       0.29      0.33      0.31       227
          10       0.15      0.05      0.08       455
          11       0.25      0.01      0.02      1660
          12       0.48      0.43      0.45      1261
          13       0.26      0.07      0.11       502
          14       0.00      0.00      0.00       156
          15       0.12      0.27      0.16       798
          16       0.59      0.40      0.48   

## **With MFCCs and CQT**

In [53]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(42578, 9048)

In [54]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(13083, 9048)

In [55]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [56]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2546816479400749


In [57]:
val_scores['mfcc_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.18      0.34      0.24       793
           1       0.47      0.53      0.50      1294
           2       0.08      0.35      0.14       617
           3       0.34      0.35      0.35       984
           4       0.04      0.03      0.03       229
           5       0.01      0.01      0.01       303
           6       0.00      0.00      0.00       275
           7       0.31      0.42      0.35       698
           8       0.19      0.46      0.27       176
           9       0.22      0.24      0.23       227
          10       0.22      0.12      0.15       455
          11       0.23      0.01      0.02      1660
          12       0.40      0.38      0.39      1261
          13       0.26      0.05      0.08       502
          14       0.01      0.01      0.01       156
          15       0.16      0.17      0.17       798
          16       0.78      0.59      0.67   

## **With Chroma and CQT**

In [58]:
training_features_3D = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(42578, 8352)

In [59]:
validation_features_3D = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(13083, 8352)

In [60]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [61]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma_cqt'] = rf.score(trainingv _features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.19559734005961935


In [62]:
val_scores['chroma_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.07      0.13      0.09       793
           1       0.44      0.44      0.44      1294
           2       0.10      0.30      0.14       617
           3       0.15      0.24      0.18       984
           4       0.00      0.00      0.00       229
           5       0.00      0.00      0.00       303
           6       0.00      0.00      0.00       275
           7       0.25      0.44      0.32       698
           8       0.18      0.40      0.25       176
           9       0.02      0.02      0.02       227
          10       0.05      0.02      0.02       455
          11       0.04      0.00      0.00      1660
          12       0.30      0.30      0.30      1261
          13       0.28      0.04      0.07       502
          14       0.01      0.03      0.02       156
          15       0.09      0.12      0.10       798
          16       0.63      0.48      0.55   

## **With Mel-Spectrogram and Chroma**

In [63]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(42578, 12180)

In [64]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(13083, 12180)

In [65]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [66]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.22334326989222655


In [67]:
val_scores['melspectrogram_chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.14      0.28      0.19       793
           1       0.38      0.51      0.43      1294
           2       0.15      0.28      0.19       617
           3       0.22      0.28      0.24       984
           4       0.00      0.00      0.00       229
           5       0.00      0.00      0.00       303
           6       0.00      0.00      0.00       275
           7       0.34      0.38      0.36       698
           8       0.16      0.52      0.25       176
           9       0.06      0.20      0.09       227
          10       0.33      0.18      0.24       455
          11       0.18      0.00      0.01      1660
          12       0.32      0.21      0.25      1261
          13       0.54      0.15      0.24       502
          14       0.03      0.07      0.04       156
          15       0.15      0.28      0.19       798
          16       0.50      0.39      0.44   

## **With Mel-Spectrogram and CQT**

In [68]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(42578, 18444)

In [69]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(13083, 18444)

In [70]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [71]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.23412061453795002


In [72]:
val_scores['melspectrogram_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.11      0.19      0.14       793
           1       0.45      0.51      0.48      1294
           2       0.14      0.32      0.20       617
           3       0.17      0.27      0.21       984
           4       0.07      0.01      0.02       229
           5       0.00      0.00      0.00       303
           6       0.00      0.00      0.00       275
           7       0.37      0.40      0.38       698
           8       0.15      0.53      0.23       176
           9       0.08      0.25      0.12       227
          10       0.19      0.08      0.11       455
          11       0.20      0.01      0.01      1660
          12       0.41      0.31      0.36      1261
          13       0.38      0.07      0.11       502
          14       0.04      0.10      0.06       156
          15       0.13      0.21      0.16       798
          16       0.62      0.61      0.61   

## **With Mel-Spectrogram, MFCCs, CQT**

In [73]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(42578, 20184)

In [74]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(13083, 20184)

In [75]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [76]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.24703814109913627


In [77]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.14      0.27      0.18       793
           1       0.44      0.54      0.49      1294
           2       0.12      0.32      0.18       617
           3       0.25      0.31      0.27       984
           4       0.05      0.01      0.02       229
           5       0.00      0.00      0.00       303
           6       0.00      0.00      0.00       275
           7       0.32      0.36      0.34       698
           8       0.14      0.55      0.22       176
           9       0.09      0.25      0.14       227
          10       0.21      0.11      0.14       455
          11       0.39      0.01      0.02      1660
          12       0.45      0.34      0.39      1261
          13       0.29      0.04      0.07       502
          14       0.03      0.08      0.05       156
          15       0.12      0.18      0.14       798
          16       0.69      0.63      0.66   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [78]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(42578, 13920)

In [79]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(13083, 13920)

In [80]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [81]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.24841397233050524


In [82]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.20      0.40      0.26       793
           1       0.43      0.52      0.47      1294
           2       0.13      0.30      0.18       617
           3       0.25      0.32      0.28       984
           4       0.04      0.01      0.02       229
           5       0.00      0.00      0.00       303
           6       0.00      0.00      0.00       275
           7       0.31      0.36      0.33       698
           8       0.16      0.53      0.24       176
           9       0.08      0.23      0.12       227
          10       0.30      0.19      0.23       455
          11       0.52      0.01      0.02      1660
          12       0.44      0.30      0.36      1261
          13       0.33      0.07      0.12       502
          14       0.03      0.07      0.04       156
          15       0.15      0.24      0.18       798
          16       0.60      0.49      0.54   

## Review results from all models

In [83]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.251
3,chroma,1.0,0.209
8,chroma_cqt,1.0,0.196
4,cqt,1.0,0.188
2,melspectrogram,1.0,0.226
9,melspectrogram_chroma,1.0,0.223
12,melspectrogram_chroma_mfcc,1.0,0.248
10,melspectrogram_cqt,1.0,0.234
11,melspectrogram_cqt_mfcc,1.0,0.247
1,mfcc,1.0,0.317


In [84]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(2)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features      0.23       0.74
3                    chroma      0.17       0.68
8                chroma_cqt      0.18       0.67
4                       cqt      0.17       0.66
2            melspectrogram      0.21       0.72
9     melspectrogram_chroma      0.21       0.72
10       melspectrogram_cqt      0.22       0.71
11  melspectrogram_cqt_mfcc      0.23       0.75
1                      mfcc      0.29       0.79
6               mfcc_chroma      0.29       0.78
7                  mfcc_cqt      0.24       0.74
5       mfcc_melspectrogram      0.23       0.74
